In [1]:
import pandas as pd

In [2]:
data = pd.concat([pd.read_csv("data_for_EDA.csv")])

In [3]:
del data['Unnamed: 0']

In [7]:
data['total reward per total amount'] = data['reward']/data['Total_amount']

In [8]:
data['Total_amount'] = data['Total_amount'].apply(lambda x : .00001 if x==0 else x)

In [9]:
data['time'] = data['time'].apply(lambda x : .0001 if x ==0 else x)

In [10]:
data['Total amount spent on website per month'] = data['Total_amount']*24*30/data['time']

In [11]:
data['Total reward got from website per month'] = data['reward']*24*30/data['time']

In [12]:
data['Total_offer_received'] = data['Total_offer_received'].apply(lambda x : .0001 if x ==0 else x)

In [13]:
data['Total viewed offer per Total sent offer'] = data['Total_view']/data['Total_offer_received']

In [14]:
data['Total completed offer per Total sent offer'] = data['offer completed']/data['Total_offer_received']

In [15]:
data['Total transaction made per month'] = data['total_transaction_made']*24*30/data['time']

In [16]:
data = data[['gender',
      'age',
      'income',
      'Year', #Year, when customer was acquired
      'month', #Year, when customer was acquired
      'web',
      'email', 
      'mobile', 
      'social',
     'offer_type',
      'duration',
      'difficulty',
      'reward',
      'week_day_in_which_offer_is_made',
      'total reward per total amount',
      'Total amount spent on website per month',
     'Total reward got from website per month',
      'Total viewed offer per Total sent offer',
     'Total completed offer per Total sent offer',
     'Total transaction made per month',
      'is_offer_completed'      
            ]]

In [19]:
for k in ['Year','month','week_day_in_which_offer_is_made']:
    data[k] = data[k].apply(lambda x : str(x))

C:\Users\puru\Miniconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [20]:
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score,roc_auc_score,confusion_matrix,precision_score,f1_score,recall_score

from imblearn.over_sampling import SMOTE

from sklearn.preprocessing import StandardScaler,MinMaxScaler,RobustScaler

import pandas as pd, numpy as np,pymysql,datetime as dt

from sklearn.feature_selection import VarianceThreshold

from imblearn.pipeline import make_pipeline

In [21]:
pipe = make_pipeline( RandomForestClassifier(random_state=100,n_estimators= 200) )

In [22]:
from sklearn.model_selection import GridSearchCV

In [23]:
gsc = GridSearchCV(   estimator=pipe,
    
                      param_grid={
                          
                          'randomforestclassifier__max_depth':[1,2,3,4,5,6,7,8,9,10],
                                  },
                      scoring='f1',
                      
                      cv= 5)

In [24]:
import warnings

warnings.filterwarnings("ignore")

In [25]:
data = pd.get_dummies(data)

In [26]:
data = data.fillna(data.median())

In [27]:
for k in data.columns:
    
    data[k] = data[k].apply(lambda x : 0 if x ==  np.inf else x)

In [28]:
columns = data.columns

In [29]:
from sklearn.preprocessing import RobustScaler,StandardScaler

In [30]:
scaler = StandardScaler()

In [31]:
data = pd.DataFrame(data)

In [32]:
data.columns = columns

In [33]:
y = data.is_offer_completed

X = data.drop("is_offer_completed",axis = 1)

data = scaler.fit_transform(data)

x_train, x_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state =1, stratify =y)

In [34]:
 gsc.fit(x_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('randomforestclassifier', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
            oob_score=False, random_state=100, verbose=0, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'randomforestclassifier__max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1', verbose=0)

In [36]:
y_pred = gsc.best_estimator_.predict(x_test)

In [37]:
f1_score(y_pred,y_test),precision_score(y_pred,y_test),recall_score(y_pred,y_test)

(0.7732580325116386, 0.8013286934514394, 0.7470874502285798)

In [38]:
t = gsc.best_estimator_

In [39]:
max_dept = dict(t.steps)['randomforestclassifier'].max_depth

In [40]:
features = dict(t.steps)['randomforestclassifier'].feature_importances_

In [41]:
f = x_train.columns

In [42]:
from sklearn.feature_selection import SelectFromModel

In [48]:
sel_ = SelectFromModel(RandomForestClassifier(n_estimators=200,random_state=1,max_depth = max_dept),threshold="mean")

In [49]:
sel_.fit(x_train,y_train)

SelectFromModel(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
            oob_score=False, random_state=1, verbose=0, warm_start=False),
        norm_order=1, prefit=False, threshold='mean')

In [50]:
x_train.columns[(sel_.get_support())]

Index(['age', 'income', 'reward', 'total reward per total amount',
       'Total amount spent on website per month',
       'Total reward got from website per month',
       'Total completed offer per Total sent offer',
       'Total transaction made per month', 'gender_F', 'Year_2016',
       'Year_2018'],
      dtype='object')

In [51]:
X_train_rf = pd.DataFrame(sel_.transform(x_train))
    
X_test_rf = pd.DataFrame(sel_.transform(x_test))

In [52]:
gsc2 = GridSearchCV(
        
    estimator=pipe,
    
    param_grid={

        
      'randomforestclassifier__max_depth':[1,2,3,4,5,6,7],
        
    },
    
    scoring='f1',
    cv=3 )
    
gsc2.fit(X_train_rf, y_train)
    
y_pred = gsc2.predict(X_test_rf)

f1_score(y_pred,y_test),precision_score(y_pred,y_test),recall_score(y_pred,y_test)

(0.7633751146438399, 0.7899398924391016, 0.7385388938183969)

In [53]:
from sklearn.tree import DecisionTreeClassifier

In [54]:
pipe = make_pipeline( DecisionTreeClassifier(random_state=100))

In [55]:
gsc2 = GridSearchCV(
        
    estimator=pipe,
    
    param_grid={

        
      'decisiontreeclassifier__max_depth':[1,2,3,4,5,6,7,8,9,10],
        
    },
    
    scoring='f1',
    cv=3 )
    
gsc2.fit(X_train_rf, y_train)
    
y_pred = gsc2.predict(X_test_rf)

f1_score(y_pred,y_test),precision_score(y_pred,y_test),recall_score(y_pred,y_test),accuracy_score(y_pred,y_test)

(0.7597572851899017,
 0.8021195824106295,
 0.7216450832503202,
 0.7373249242362192)

In [56]:
from sklearn.linear_model import LogisticRegression

In [57]:
pipe = make_pipeline( LogisticRegression(random_state=100))
gsc2 = GridSearchCV(
        
    estimator=pipe,
    
    param_grid={

        
      'logisticregression__penalty':['l2','l1'],
        
    },
    
    scoring='f1',
    
    cv=3 
                  )
    
gsc2.fit(X_train_rf, y_train)
    
y_pred = gsc2.predict(X_test_rf)

f1_score(y_pred,y_test),precision_score(y_pred,y_test),recall_score(y_pred,y_test),accuracy_score(y_pred,y_test)

(0.7242061920975532,
 0.7233470420752927,
 0.7250673854447439,
 0.7147186501760996)

In [58]:
from sklearn.svm import SVC

In [59]:
svm = SVC(random_state=1)

In [60]:
svm.fit(X_train_rf, y_train)
    
y_pred = svm.predict(X_test_rf)

f1_score(y_pred,y_test),precision_score(y_pred,y_test),recall_score(y_pred,y_test),accuracy_score(y_pred,y_test)

(0.7152526779260038,
 0.8608035431825372,
 0.6118043844856661,
 0.6450978786141371)

In [61]:
f1_score(y_pred,y_test),precision_score(y_pred,y_test),recall_score(y_pred,y_test),accuracy_score(y_pred,y_test)

(0.7152526779260038,
 0.8608035431825372,
 0.6118043844856661,
 0.6450978786141371)